In [1]:
from __future__ import print_function
import sys
import cv2
import os
import math
import glob
import hashlib
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
import shutil
import site
from PIL import Image
from tqdm.notebook import tqdm
from datetime import datetime
from multiprocessing import Pool
from moviepy.editor import *

%matplotlib inline


In [2]:
#print('numpy:', np.__version__)
#print('pandas:', pd.__version__)
#print('matplotlib:', mplv)
print('OpenCV:', cv2.__version__)
print(list(map(lambda x: x.replace("\\", "/"), site.getsitepackages())))

OpenCV: 4.4.0
['C:/Users/eiuru/anaconda3/envs/py38', 'C:/Users/eiuru/anaconda3/envs/py38/lib/site-packages']


In [3]:
CLIP_TARGET_FOLDER_NAMES = ["s1", "s2"]
CHARACTOR_NAMES = ["sayo", "tsugumi", "tae", "kanon", "hina", "aya", "eve"]

In [4]:
def list_files(dir):
    return [os.path.join(dir, p.name) for p in os.scandir(dir)]

def get_filename_without_ext(p):
    return os.path.splitext(os.path.basename(p))[0]

def get_filename_andext(p):
    return os.path.splitext(os.path.basename(p))

In [5]:
def clip_movie(movie_file_path, clips, dst):
    video = VideoFileClip(movie_file_path)
    print(movie_file_path, clips, dst)
    clipsArray = [] 
    for clip in clips:
        [startTime, endTime] = list(map(int, clip.split('-')))
        print(startTime, endTime)
        clip = video.subclip(startTime / video.fps, endTime / video.fps)
        clipsArray.append(clip)
    final = concatenate_videoclips(clipsArray)
    final.write_videofile(dst, fps=video.fps, codec='libx264', audio_codec="aac")
    

In [6]:
def main():
    cwd = os.getcwd()
    movie_dir = os.path.join(cwd, 'movies')
    
    for series_name in CLIP_TARGET_FOLDER_NAMES:
        for charactor_name in CHARACTOR_NAMES:
            movies = list_files(os.path.join(movie_dir, series_name))
            record_dir = os.path.join(cwd, "02.clip", "records", charactor_name, series_name)
            records = list_files(record_dir)

            crop_dir = os.path.join(cwd, 'crop_movies', charactor_name, series_name)
            if not os.path.exists(crop_dir):
                os.makedirs(crop_dir, exist_ok=True)

            tmp_dir = os.path.join(cwd, 'crop_tmp', charactor_name, series_name)
            if not os.path.exists(tmp_dir):
                os.makedirs(tmp_dir, exist_ok=True)

            pbar = tqdm(records)
            for p in pbar:
                with open(p, encoding='UTF-8') as f:
                    lines = f.readlines()
                    movie_raw_file_path = lines[0].strip()
                    movie_encoded_file_name = lines[1].strip()
                    clips = list(map(lambda x: x.strip(), lines[2:]))
                    if len(clips) == 0: 
                        pbar.update(1)
                        continue
                    dst = os.path.join(crop_dir, movie_encoded_file_name)
                    if os.path.exists(dst): 
                        pbar.update(1)
                        continue
                    tmp_file_path = os.path.join(tmp_dir, movie_encoded_file_name)
                    shutil.copy2(movie_raw_file_path, tmp_file_path)
                    clip_movie(tmp_file_path, clips, dst)

if __name__ == "__main__":
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    main()
    

2020-12-29 02:49:24


FileNotFoundError: [WinError 3] 指定されたパスが見つかりません。: 'F:\\workspace\\Github\\development_projects\\00_developing\\sayo-voice\\02.clip\\records\\sayo\\s2'